# YoloV3 TF2 GPU Colab Notebook

##### 1.  Clone and install dependencies 

**IMPORTANT**: Restart following the instruction

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!git clone https://github.com/zzh8829/yolov3-tf2
%cd /content/yolov3-tf2/
!pip install -r requirements-gpu.txt

Cloning into 'yolov3-tf2'...
remote: Enumerating objects: 410, done.
remote: Total 410 (delta 0), reused 0 (delta 0), pack-reused 410
Receiving objects: 100% (410/410), 4.23 MiB | 29.28 MiB/s, done.
Resolving deltas: 100% (236/236), done.
/content/yolov3-tf2
Obtaining file:///content/yolov3-tf2 (from -r requirements-gpu.txt (line 6))
  Found existing installation: yolov3-tf2 0.1
    Can't uninstall 'yolov3-tf2'. No files were found to uninstall.
  Running setup.py develop for yolov3-tf2


##### 2.  Check Tensorflow2 version

In [ ]:
%cd /content/yolov3-tf2/
!ls

import tensorflow as tf
tf.__version__

/content/yolov3-tf2
checkpoints	 data		  README.md		train.py
colab_gpu.ipynb  detect.py	  requirements-gpu.txt	yolov3_tf2
conda-cpu.yml	 detect_video.py  requirements.txt	yolov3-tf2
conda-gpu.yml	 docs		  setup.py		yolov3_tf2.egg-info
convert.py	 LICENSE	  tools


'2.4.0'

##### 3.  Convert Pretrained Darknet Weight

In [ ]:
#yolov3
!wget https://pjreddie.com/media/files/yolov3.weights -O data/yolov3.weights

%cd /content/yolov3-tf2
!python convert.py --weights ./data/yolov3.weights --output ./data/yolov3.tf

--2021-05-17 14:34:36--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘data/yolov3.weights’

data/yolov3.weights 100%[===================>] 236.52M  39.7MB/s    in 6.4s    

2021-05-17 14:34:43 (36.8 MB/s) - ‘data/yolov3.weights’ saved [248007048/248007048]

/content/yolov3-tf2
2021-05-17 14:34:44.260228: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-17 14:34:46.314804: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-17 14:34:46.345442: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-05-17 14:34:46.426463: E tensorflow/stream_executo

In [ ]:
# yolov3-tiny
! wget https://pjreddie.com/media/files/yolov3-tiny.weights -O data/yolov3-tiny.weights
! python convert.py --weights ./data/yolov3-tiny.weights --output ./checkpoints/yolov3-tiny.tf --tiny

--2021-05-08 12:47:49--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘data/yolov3-tiny.weights’

data/yolov3-tiny.we 100%[===================>]  33.79M  14.1MB/s    in 2.4s    

2021-05-08 12:47:52 (14.1 MB/s) - ‘data/yolov3-tiny.weights’ saved [35434956/35434956]

2021-05-08 12:47:53.433052: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-08 12:47:55.298559: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-08 12:47:55.299351: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-05-08 12:47:55.314764: I tensorflow/stream_executor/cuda/cu

# 7. My training

inspired from https://machinelearningspace.com/yolov3-tensorflow-2-part-3/

In [ ]:
import numpy as np
import cv2
import time
import os
import matplotlib.pyplot as plt
import pandas as pd
# keras
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, Input, ZeroPadding2D, LeakyReLU, UpSampling2D

import xml.etree.ElementTree as ET


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
code_dir='/content/drive/My Drive/MSc Computational Neuroscience, Cognition & AI/MLis2/ML Project/code'

# test images
dataset_dir= os.path.join(code_dir,'data/test_data')
images_dir=os.path.join(dataset_dir, 'test_data')
images_list = os.listdir(images_dir)
print(images_list[:20])


['1009.png', '1016.png', '1017.png', '1013.png', '1001.png', '1019.png', '101.png', '1018.png', '1012.png', '1002.png', '128.png', '12.png', '110.png', '122.png', '123.png', '114.png', '129.png', '111.png', '125.png', '113.png']


### Data preprocessing

upload traffic_train.tfrecord, traffic_val.tfrecord, traffic.names


In [ ]:
# yolov3

import sys
from absl import app, logging, flags
from absl.flags import FLAGS
import time
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import YoloV3
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs

flags.DEFINE_string('classes', './data/traffic.names', 'path to classes file')
flags.DEFINE_string('weights', './data/yolov3.tf',
                    'path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('image', './data/girl.png', 'path to input image')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_string('output', './output.jpg', 'path to output image')
flags.DEFINE_integer('num_classes', 8, 'number of classes in the model')

app._run_init(['yolov3'], app.parse_flags_with_usage)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# yolov3-tiny

import sys
from absl import app, logging, flags
from absl.flags import FLAGS
import time
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import YoloV3Tiny
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs

flags.DEFINE_string('classes', './data/traffic.names', 'path to classes file')
flags.DEFINE_string('weights', './checkpoints/yolov3-tiny.tf',
                    'path to weights file')
flags.DEFINE_boolean('tiny', True, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 320, 'resize images to')
flags.DEFINE_string('image', './data/girl.png', 'path to input image')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_string('output', './output.jpg', 'path to output image')
flags.DEFINE_integer('num_classes', 8, 'number of classes in the model')

app._run_init(['yolov3-tiny'], app.parse_flags_with_usage)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# yolov3-tiny
! python detect.py --weights ./checkpoints/yolov3-tiny.tf --tiny --image ./data/street.jpg


2021-05-08 12:48:17.249868: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-08 12:48:19.092641: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-08 12:48:19.093491: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-05-08 12:48:19.098187: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-08 12:48:19.099152: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-05-08 12:48:19.099210: I tensorflow/stream_executor/platform/def

### train

import traffic_train.tfrecord, traffic_val.tfrecord, traffic.names (txt file names by order of class)

In [ ]:
!python tools/visualize_dataset.py --dataset ./data/traffic_train.tfrecord --classes=./data/traffic.names


2021-05-08 12:48:26.874451: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
I0508 12:48:28.712275 139886250841984 visualize_dataset.py:22] classes loaded
2021-05-08 12:48:28.712724: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-08 12:48:28.713536: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-05-08 12:48:28.718553: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-08 12:48:28.719554: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681

  --mode eager_tf --transfer fine_tune \


In [ ]:
# yolov3

!python train.py \
	--dataset ./data/traffic_train.tfrecord \
	--val_dataset ./data/traffic_val.tfrecord \
	--classes ./data/traffic.names \
	--num_classes 8 \
	--mode fit --transfer darknet \
	--batch_size 16 \
	--epochs 30\
	--weights ./data/yolov3.tf \
	--weights_num_classes 80 

In [ ]:
# yolov3-tiny

!python train.py \
	--dataset ./data/traffic_train.tfrecord \
	--val_dataset ./data/traffic_val.tfrecord \
	--classes ./data/traffic.names \
	--num_classes 8 \
	--mode fit --transfer darknet \
  --size 320 \
  --tiny 'true' \
	--batch_size 28 \
	--epochs 30\
	--weights ./checkpoints/yolov3-tiny.tf \
	--weights_num_classes 80 

2021-05-08 15:50:27.415065: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-08 15:50:29.557958: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-08 15:50:29.558883: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-05-08 15:50:29.564248: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-08 15:50:29.564836: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-05-08 15:50:29.564872: I tensorflow/stream_executor/platform/def